In [9]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import PROJECT.util as f
from matplotlib.pyplot import cm

In [10]:
# Pour des vecteurs u* et v*,
# les overlaps obtenus sont soit les deux positifs ou les deux négatifs, on a:
# overlap(u*)= -x et overlap(v*)= -y ou overlap(u*)= x et overlap(v*)= y.
# on prend donc le min(u-x, u+x):

# si u est positif et x est positif:
# min(u-x, u+x) -> u-x est bon

# si u est positif et x est négatif:
# min(u-(-x), u+(-x)) -> u+(-x) =u-x est bon

# si u est négatif et x est positif:
# min(-u-x, -u+x) -> -u+x =x-u est bon

# si u est négatif et x est négatif:
# min(-u-(-x), -u+(-x)) -> -u-(-x) =x-u est bon

def mse1(u,u_n):
    ## u-u_n
    a=torch.pow((u-u_n),2).mean(0)
    b=torch.pow((u+u_n),2).mean(0)
    return torch.min(a,b)
def mse2(u_e, v_e, u_t, v_t):
    ## u*v*-uv
    tmp = torch.tensordot(u_e, v_e, 0) - torch.tensordot(u_t, v_t, 0)
    return torch.sum(torch.pow((tmp), 2))
def mse3(u_e, v_e, u_t, v_t, Y):
    ## Y-uv
    tmp = Y - torch.tensordot(u_t, v_t, 0)
    return torch.sum(torch.pow((tmp), 2))

In [11]:
def main_gradient_2_sans_proj_normalisation(iteration, u_p, v_p, Y, u_, v_, N, M, lambda_, beta_u, beta_v, lambda_1, lambda_2, dt):

    res = []
    res_mse= []
    
    mod = iteration*0.1
    
    stabilisation_check=np.empty((0,2))

    for i in range(iteration):

        # Computation
        sqrt_dt = torch.sqrt(dt)

        u_1 = (1/lambda_1) * f.gradient_u_2(N, M, u_p,v_p,Y,lambda_) * dt
        u_2 = torch.sqrt(2/(lambda_1*beta_u)) * torch.empty(N).normal_(mean=0,std=sqrt_dt)
        u_3 = ((N-1)/(N*lambda_1*beta_u))*u_p*dt
        u_n = u_p - u_1 + u_2 - u_3

        v_1 = 1/lambda_2 * f.gradient_v_2(N,M,u_p,v_p,Y,lambda_) * dt
        v_2 = torch.sqrt(2/(lambda_2*beta_v)) * torch.empty(M).normal_(mean=0,std=sqrt_dt)
        v_3 = ((M-1)/(M*lambda_2*beta_v))*v_p*dt
        v_n = v_p - v_1 + v_2 - v_3

        # Normalisation
        u_n = u_n / torch.linalg.norm(u_n)
        v_n = v_n / torch.linalg.norm(v_n)
        u_n = u_n * torch.sqrt(N)
        v_n = v_n * torch.sqrt(M)

        # Re-asign for the loop
        u_p = u_n
        v_p = v_n
    
        res_u = torch.abs(f.overlap(u_,u_n,N))
        res_v = torch.abs(f.overlap(v_,v_n,M))
        res.append((res_u,res_v))
        
        res_mse.append((mse1(u_,u_n), mse1(v_,v_n)))
        
        if len(stabilisation_check)==0:
            #print("INIT")
            stabilisation_check=np.append(stabilisation_check, [[res_u,res_v]], axis=0)
        elif len(stabilisation_check)==200:
            #print("FINISH")
            print(f"overlap(u, v): ({res_u}; {res_v})")
            return np.array(res), np.array(res_mse)
        elif (np.abs(stabilisation_check-[res_u,res_v])<0.0001).all():
            #print("ADD")
            stabilisation_check=np.append(stabilisation_check, [[res_u,res_v]], axis=0)
        else:
            #print("RESET")
            stabilisation_check=np.empty((0,2))
        
        if i%mod==0:
            print(f"progress {100*i/iteration:.2f}%", end="\r")
        if i==iteration-1:
            print(f"overlap(u, v): ({res_u}; {res_v})")

    return np.array(res), np.array(res_mse)

In [12]:
def one_run(lambda_=2, N=500, M=500, beta_u=float("inf"), beta_v=float("inf"), lambda_1=1,
        lambda_2=1, dt=1/100, iteration=1000):
      #Size of the matrix Y
    N = torch.tensor(N)
    M = torch.tensor(M)

    # lambda
    lambda_ = torch.tensor(lambda_)

    #temperatures
    beta_u = torch.tensor(beta_u)
    beta_v = torch.tensor(beta_v)

    #learning rates
    lambda_1 = torch.tensor(lambda_1)
    lambda_2 = torch.tensor(lambda_2)

    # Pas de temps
    dt = torch.tensor(dt)

    u_ = f.generate_vector(N)
    v_ = f.generate_vector(M)

    Y = f.generate_Y(N, M, u_,v_, lambda_)
    #print(Y)

    # Conditions initiales
    u_p = f.generate_vector(N)
    v_p = f.generate_vector(M)

    # Perform the gradient descent
    (res, res_mse) = main_gradient_2_sans_proj_normalisation(iteration, u_p, v_p, Y, u_, v_, N, M, lambda_, beta_u, beta_v, lambda_1, lambda_2, dt)
    
    return res, res_mse


In [13]:
# MAIN METHOD:

def main(size_of_one_sample, list_value_1, list_value_2):
    
    all_overlap_iterations=[]
    all_final_overlaps=[]
    
    all_mse_iterations=[]
    all_final_mse=[]
    
    for value_to_test_1 in list_value_1:
        
        for value_to_test_2 in list_value_2:
        
            one_run_overlap_iterations=[]
            one_run_mse_iterations=[]

            # Calculation
            for _ in range(size_of_one_sample):
                (current_uv_overlap,current_uv_mse)=one_run(beta_u=value_to_test_1, beta_v=value_to_test_2)
                # Store the overlap at the end of each iterations:
                one_run_overlap_iterations.append(current_uv_overlap)
                # Store the mse at the end of each iterations:
                one_run_mse_iterations.append(current_uv_mse)

            # Store the overlap/mse at the final iteration:
            one_run_final_overlap = np.array([x[-1] for x in one_run_overlap_iterations])
            one_run_final_mse = np.array([x[-1] for x in one_run_mse_iterations])

            # Add the results of this specific run to the global array:
            # overlap at all iterations:
            all_overlap_iterations.append(one_run_overlap_iterations)
            # overlap at the final iteration:
            all_final_overlaps.append(one_run_final_overlap)
            # mse at all iterations:
            all_mse_iterations.append(one_run_mse_iterations)
            # mse at the final iteration:
            all_final_mse.append(one_run_final_mse)
            
    return np.array(all_overlap_iterations), np.array(all_final_overlaps), np.array(all_mse_iterations), np.array(all_final_mse)

In [ ]:
# MAIN CELL: DO THE COMPUTATION

#list_value_1 = np.logspace(-1,1)
list_value_1 = np.arange(0.1, 1, 0.1)
#list_value_1 = np.array([2,1,0.5])

#list_value_2 = np.logspace(-1,1)
list_value_2 = np.arange(0.1, 1, 0.1)
#list_value_2 = np.array([2,1,0.5])

size_of_one_sample=10

(overlap_at_each_iterations, overlap_at_final_iteration,
 mse_at_each_iterations, mse_at_final_iteration) = main(size_of_one_sample, list_value_1, list_value_2)

overlap(u, v): (0.05669474974274635; 0.007194702047854662)
overlap(u, v): (0.013085486367344856; 0.0682973712682724)
overlap(u, v): (0.04054351523518562; 0.03697679936885834)
overlap(u, v): (0.093378446996212; 0.06341053545475006)
overlap(u, v): (0.09279833734035492; 0.05280975624918938)
overlap(u, v): (0.009030895307660103; 0.02961196005344391)
overlap(u, v): (0.020963160321116447; 0.001858814270235598)
overlap(u, v): (0.08468328416347504; 0.01610582508146763)
overlap(u, v): (0.02094062604010105; 0.00031518362811766565)
overlap(u, v): (0.03696708381175995; 0.011114340275526047)
overlap(u, v): (0.06559333205223083; 0.005952815059572458)
overlap(u, v): (0.0048248041421175; 0.028647568076848984)
overlap(u, v): (0.042801130563020706; 0.01672704890370369)
overlap(u, v): (0.08114685863256454; 0.02934337966144085)
overlap(u, v): (0.09754276275634766; 0.03402980789542198)
overlap(u, v): (0.005933013744652271; 0.05811505764722824)
overlap(u, v): (0.024589581415057182; 0.0023819045163691044)
ov

In [ ]:
name_value_1="beta_u"
name_value_2="beta_v"

In [ ]:
# GRAPH STABILISATION DES OVERLAPS
fig, axs = plt.subplots(len(list_value_1)*len(list_value_2), figsize=(20,200), facecolor='white')
fig.tight_layout()

for i in range(len(list_value_1)*len(list_value_2)):
    axs[i].grid()
    axs[i].set_ylabel("overlap")
    axs[i].set_title(f"{name_value_1}={list_value_1[i//len(list_value_2)]}"
                    + f" & {name_value_2}={list_value_2[i%len(list_value_2)]}")
    for run_in_on_sample in range(size_of_one_sample):
        axs[i].plot(overlap_at_each_iterations[i][run_in_on_sample])

In [ ]:
# GRAPH STABILISATION DES MEAN SQUARRE ERRORS
fig, axs = plt.subplots(len(list_value_1)*len(list_value_2), figsize=(20,200), facecolor='white')
fig.tight_layout()

for i in range(len(list_value_1)*len(list_value_2)):
    axs[i].grid()
    axs[i].set_ylabel("mse")
    axs[i].set_title(f"{name_value_1}={list_value_1[i//len(list_value_2)]}"
                    + f" & {name_value_2}={list_value_2[i%len(list_value_2)]}")
    for run_in_on_sample in range(size_of_one_sample):
        axs[i].plot(mse_at_each_iterations[i][run_in_on_sample])

In [ ]:
# PRINT THE OVERLAP IN FUNCTION OF THE STEP

# On calcul la moyenne des overlaps à chaque itérations dans un même sample avec:
# overlap_at_each_iterations.mean(axis=1)
#
# ~ moyenne_sample_1 = run_1_s1 + run_2_s2 + ... / size_of_one_sample
# ~ moyenne_sample_2 = run_1_s2 + run_2_s2 + ... / size_of_one_sample
# ...

# On calcul la moyenne des overlaps u et v à chaque itérations pour chaque sample avec:
# overlap_at_each_iterations.mean(axis=1)
#
# ~ moyenne_sample_1_uv = (moyenne_sample_1_u + moyenne_sample_1_v)/2
# ...

overlap_at_each_iterations_per_sample=overlap_at_each_iterations.mean(axis=1).mean(axis=2)

plt.figure(facecolor='white')
plt.grid()
plt.ylabel("overlap")
plt.xlabel("iterations")
color=iter(cm.rainbow(np.linspace(0,1,len(list_value_1)*len(list_value_2))))

for i in range(len(list_value_1)*len(list_value_2)):
    c=next(color)
    plt.plot(overlap_at_each_iterations_per_sample[i], 
             label=f"{name_value_1}={list_value_1[i//len(list_value_2)]}"
            + f" & {name_value_2}={list_value_2[i%len(list_value_2)]}", color=c)
plt.legend(loc='upper right', bbox_to_anchor=(2, 1))

In [ ]:
# PRINT THE MSE IN FUNCTION OF THE STEP
# same as the cell above but with the men squarre error:

mse_at_each_iterations_per_sample=mse_at_each_iterations.mean(axis=1).mean(axis=2)

plt.figure(facecolor='white')
plt.grid()
plt.ylabel("mse")
plt.xlabel("iterations")
color=iter(cm.rainbow(np.linspace(0,1,len(list_value_1)*len(list_value_2))))

for i in range(len(list_value_1)*len(list_value_2)):
    c=next(color)
    plt.plot(mse_at_each_iterations_per_sample[i], 
             label=f"{name_value_1}={list_value_1[i//len(list_value_2)]}"
            + f" & {name_value_2}={list_value_2[i%len(list_value_2)]}", color=c)
plt.legend(loc='upper right', bbox_to_anchor=(2, 1))

In [ ]:
# CONTOUR PLOT OVERLAP
# moyenne de tous les runs dans un sample:
# overlap_at_final_iteration.mean(axis=1))
# moyenne entre u et v:
# overlap_at_final_iteration.mean(axis=1).mean(axis=1)

overlap_mean=overlap_at_final_iteration.mean(axis=1).mean(axis=1)

x, y = np.meshgrid(list_value_2,list_value_1)
overlap_mean=overlap_mean.reshape(len(list_value_1),len(list_value_2))

fig,ax=plt.subplots(1,1, facecolor='white')
c = ax.contourf(x,y,overlap_mean)
fig.colorbar(c)
plt.show()


In [ ]:
# CONTOUR PLOT MSE
# moyenne de tous les runs dans un sample:
# overlap_at_final_iteration.mean(axis=1))
# moyenne entre u et v:
# overlap_at_final_iteration.mean(axis=1).mean(axis=1)

mse_mean=mse_at_final_iteration.mean(axis=1).mean(axis=1)

x, y = np.meshgrid(list_value_2,list_value_1)
mse_mean=mse_mean.reshape(len(list_value_1),len(list_value_2))

fig,ax=plt.subplots(1,1, facecolor='white')
cmap=cm.get_cmap('rainbow')
c = ax.contourf(x,y,mse_mean,cmap=cmap)
fig.colorbar(c)
ax.set_xlabel(f"{name_value_2}")
ax.set_ylabel(f"{name_value_1}")
plt.show()